In [25]:
import struct
import numpy as np
import os
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

max_index = 295521943

In [26]:
embed_table = np.random.rand(max_index,16).astype(np.float32)

In [27]:
import tqdm

In [28]:
pipeline = r.pipeline()


In [11]:
embed_table = np.random.rand(max_index,16).astype(np.float32)

In [29]:
for i in tqdm.tqdm(range(embed_table.shape[0])):
  key = f"key_{i}"
  embed = embed_table[i]
  arr_bytes = embed.tobytes()

  pipeline.set(key, arr_bytes)
pipeline.execute()


100%|██████████| 295521943/295521943 [13:29<00:00, 365039.02it/s] 


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [24]:
%%time
# pipeline.mget(['key_0', 'key_1'])
arrays_bytes =  r.mget(['key_30590514', 'key_73618409'])
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

CPU times: user 1.04 ms, sys: 0 ns, total: 1.04 ms
Wall time: 806 µs


[]

In [30]:
batch_dim = [16,32,64,128,256,512]
buff_dict = {i:np.random.rand(1,i).astype(np.float32).tobytes() for i in batch_dim}
for i in tqdm.tqdm(range(1000)):
  for dim in batch_dim:
    key = f"key_{i}_dim_{dim}"
    embed = buff_dict[dim]

    pipeline.set(key, embed)
pipeline.execute()

100%|██████████| 1000/1000 [00:02<00:00, 453.60it/s]


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [62]:
import random


In [63]:

%%timeit
# 16

keys = [f"key_{i}_dim_{16}" for i in [random.randint(0, 700) for _ in range(10)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

107 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [64]:

%%timeit
# 16

keys = [f"key_{i}_dim_{16}" for i in [random.randint(0, 700) for _ in range(100)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

435 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [66]:

%%timeit
# 16

keys = [f"key_{i}_dim_{32}" for i in [random.randint(0, 700) for _ in range(10)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

109 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [67]:

%%timeit
# 16

keys = [f"key_{i}_dim_{32}" for i in [random.randint(0, 700) for _ in range(100)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

453 µs ± 6.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [72]:

%%timeit
# 16

keys = [f"key_{i}_dim_{64}" for i in [random.randint(0, 700) for _ in range(10)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

114 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [77]:

%%timeit
# 16

keys = [f"key_{i}_dim_{64}" for i in [random.randint(0, 700) for _ in range(100)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]


466 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [78]:
%%timeit

# 16

keys = [f"key_{i}_dim_{128}" for i in [random.randint(0, 700) for _ in range(10)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]

122 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [80]:
%%timeit

# 16

keys = [f"key_{i}_dim_{128}" for i in [random.randint(0, 700) for _ in range(100)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]
arrays

508 µs ± 6.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [81]:
%%timeit

# 16

keys = [f"key_{i}_dim_{512}" for i in [random.randint(0, 700) for _ in range(10)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]


135 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [82]:
%%timeit
# 16

keys = [f"key_{i}_dim_{512}" for i in [random.randint(0, 700) for _ in range(100)]]
arrays_bytes =  r.mget(keys)
arrays = [np.frombuffer(arr, dtype=np.float32) for arr in arrays_bytes if arr]

652 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
